In [1]:
!pip install brisque

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 5.9 MB/s eta 0:00:00
  Created wheel for libsvm-official: filename=libsvm_official-3.35.0-cp310-cp310-linux_x86_64.whl size=124211 sha256=7baab8280beb3177b3d961375d41990ad7bef18c6cae04250f474b584b20cf35
  Stored in directory: /root/.cache/pip/wheels/ec/50/a6/962c82577759a39080b2d0b51640bc1be45d02bd5d0d5dde7d
Successfully built libsvm-official


In [2]:
!git clone https://github.com/guptapraful/niqe.git
%cd /kaggle/working/niqe

Cloning into 'niqe'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 31 (delta 7), reused 19 (delta 7), pack-reused 12 (from 1)
Receiving objects: 100% (31/31), 2.53 MiB | 23.55 MiB/s, done.
Resolving deltas: 100% (7/7), done.
/kaggle/working/niqe


In [3]:
import numpy as np
import scipy.misc
import scipy.io
from os.path import dirname
from os.path import join
import scipy
from PIL import Image
import numpy as np
import scipy.ndimage
import numpy as np
import scipy.special
import math
import os
from skimage import io, img_as_ubyte
from skimage.color import rgb2gray
from brisque import BRISQUE

gamma_range = np.arange(0.2, 10, 0.001)
a = scipy.special.gamma(2.0/gamma_range)
a *= a
b = scipy.special.gamma(1.0/gamma_range)
c = scipy.special.gamma(3.0/gamma_range)
prec_gammas = a/(b*c)

def aggd_features(imdata):
    #flatten imdata
    imdata.shape = (len(imdata.flat),)
    imdata2 = imdata*imdata
    left_data = imdata2[imdata<0]
    right_data = imdata2[imdata>=0]
    left_mean_sqrt = 0
    right_mean_sqrt = 0
    if len(left_data) > 0:
        left_mean_sqrt = np.sqrt(np.average(left_data))
    if len(right_data) > 0:
        right_mean_sqrt = np.sqrt(np.average(right_data))

    if right_mean_sqrt != 0:
      gamma_hat = left_mean_sqrt/right_mean_sqrt
    else:
      gamma_hat = np.inf
    #solve r-hat norm

    imdata2_mean = np.mean(imdata2)
    if imdata2_mean != 0:
      r_hat = (np.average(np.abs(imdata))**2) / (np.average(imdata2))
    else:
      r_hat = np.inf
    rhat_norm = r_hat * (((math.pow(gamma_hat, 3) + 1)*(gamma_hat + 1)) / math.pow(math.pow(gamma_hat, 2) + 1, 2))

    #solve alpha by guessing values that minimize ro
    pos = np.argmin((prec_gammas - rhat_norm)**2);
    alpha = gamma_range[pos]

    gam1 = scipy.special.gamma(1.0/alpha)
    gam2 = scipy.special.gamma(2.0/alpha)
    gam3 = scipy.special.gamma(3.0/alpha)

    aggdratio = np.sqrt(gam1) / np.sqrt(gam3)
    bl = aggdratio * left_mean_sqrt
    br = aggdratio * right_mean_sqrt

    #mean parameter
    N = (br - bl)*(gam2 / gam1)#*aggdratio
    return (alpha, N, bl, br, left_mean_sqrt, right_mean_sqrt)

def ggd_features(imdata):
    nr_gam = 1/prec_gammas
    sigma_sq = np.var(imdata)
    E = np.mean(np.abs(imdata))
    rho = sigma_sq/E**2
    pos = np.argmin(np.abs(nr_gam - rho));
    return gamma_range[pos], sigma_sq

def paired_product(new_im):
    shift1 = np.roll(new_im.copy(), 1, axis=1)
    shift2 = np.roll(new_im.copy(), 1, axis=0)
    shift3 = np.roll(np.roll(new_im.copy(), 1, axis=0), 1, axis=1)
    shift4 = np.roll(np.roll(new_im.copy(), 1, axis=0), -1, axis=1)

    H_img = shift1 * new_im
    V_img = shift2 * new_im
    D1_img = shift3 * new_im
    D2_img = shift4 * new_im

    return (H_img, V_img, D1_img, D2_img)


def gen_gauss_window(lw, sigma):
    sd = np.float32(sigma)
    lw = int(lw)
    weights = [0.0] * (2 * lw + 1)
    weights[lw] = 1.0
    sum = 1.0
    sd *= sd
    for ii in range(1, lw + 1):
        tmp = np.exp(-0.5 * np.float32(ii * ii) / sd)
        weights[lw + ii] = tmp
        weights[lw - ii] = tmp
        sum += 2.0 * tmp
    for ii in range(2 * lw + 1):
        weights[ii] /= sum
    return weights

def compute_image_mscn_transform(image, C=1, avg_window=None, extend_mode='constant'):
    if avg_window is None:
      avg_window = gen_gauss_window(3, 7.0/6.0)
    assert len(np.shape(image)) == 2
    h, w = np.shape(image)
    mu_image = np.zeros((h, w), dtype=np.float32)
    var_image = np.zeros((h, w), dtype=np.float32)
    image = np.array(image).astype('float32')
    scipy.ndimage.correlate1d(image, avg_window, 0, mu_image, mode=extend_mode)
    scipy.ndimage.correlate1d(mu_image, avg_window, 1, mu_image, mode=extend_mode)
    scipy.ndimage.correlate1d(image**2, avg_window, 0, var_image, mode=extend_mode)
    scipy.ndimage.correlate1d(var_image, avg_window, 1, var_image, mode=extend_mode)
    var_image = np.sqrt(np.abs(var_image - mu_image**2))
    return (image - mu_image)/(var_image + C), var_image, mu_image


def _niqe_extract_subband_feats(mscncoefs):
    # alpha_m,  = extract_ggd_features(mscncoefs)
    alpha_m, N, bl, br, lsq, rsq = aggd_features(mscncoefs.copy())
    pps1, pps2, pps3, pps4 = paired_product(mscncoefs)
    alpha1, N1, bl1, br1, lsq1, rsq1 = aggd_features(pps1)
    alpha2, N2, bl2, br2, lsq2, rsq2 = aggd_features(pps2)
    alpha3, N3, bl3, br3, lsq3, rsq3 = aggd_features(pps3)
    alpha4, N4, bl4, br4, lsq4, rsq4 = aggd_features(pps4)
    return np.array([alpha_m, (bl+br)/2.0,
            alpha1, N1, bl1, br1,  # (V)
            alpha2, N2, bl2, br2,  # (H)
            alpha3, N3, bl3, bl3,  # (D1)
            alpha4, N4, bl4, bl4,  # (D2)
    ])

def get_patches_train_features(img, patch_size, stride=8):
    return _get_patches_generic(img, patch_size, 1, stride)

def get_patches_test_features(img, patch_size, stride=8):
    return _get_patches_generic(img, patch_size, 0, stride)

def extract_on_patches(img, patch_size):
    h, w = img.shape
    patch_size = int(patch_size)
    patches = []
    for j in range(0, h-patch_size+1, patch_size):
        for i in range(0, w-patch_size+1, patch_size):
            patch = img[j:j+patch_size, i:i+patch_size]
            patches.append(patch)

    patches = np.array(patches)
    
    patch_features = []
    for p in patches:
        patch_features.append(_niqe_extract_subband_feats(p))
    patch_features = np.array(patch_features)

    return patch_features

def _get_patches_generic(img, patch_size, is_train, stride):
    h, w = np.shape(img)
    if h < patch_size or w < patch_size:
        print("Input image is too small")
        exit(0)

    # Ensure that the patch divides evenly into img
    hoffset = (h % patch_size)
    woffset = (w % patch_size)

    if hoffset > 0: 
        img = img[:-hoffset, :]
    if woffset > 0:
        img = img[:, :-woffset]

    img = img.astype(np.float32)

    # Resize using PIL.Image.resize instead of scipy.misc.imresize
    img_pil = Image.fromarray(img)  # Convert numpy array to PIL Image
    img2_pil = img_pil.resize((int(img_pil.width * 0.5), int(img_pil.height * 0.5)), Image.BICUBIC)  # Resize by 50%
    img2 = np.array(img2_pil)  # Convert back to numpy array

    mscn1, var, mu = compute_image_mscn_transform(img)
    mscn1 = mscn1.astype(np.float32)

    mscn2, _, _ = compute_image_mscn_transform(img2)
    mscn2 = mscn2.astype(np.float32)

    feats_lvl1 = extract_on_patches(mscn1, patch_size)
    feats_lvl2 = extract_on_patches(mscn2, patch_size / 2)

    feats = np.hstack((feats_lvl1, feats_lvl2))  # feats_lvl3))

    return feats



def niqe(inputImgData, params_file_path='/kaggle/working/niqe/data/niqe_image_params.mat'):
    patch_size = 96

    # Check if the parameter file exists
    if not os.path.exists(params_file_path):
        raise FileNotFoundError(f"The parameter file was not found at: {params_file_path}")

    # Load the NIQE parameters from the provided path
    params = scipy.io.loadmat(params_file_path)
    pop_mu = np.ravel(params["pop_mu"])
    pop_cov = params["pop_cov"]

    M, N = inputImgData.shape

    # Make sure the image is large enough for the processing
    assert M > (patch_size * 2 + 1), "NIQE called with small frame size. Requires a resolution > 192x192."
    assert N > (patch_size * 2 + 1), "NIQE called with small frame size. Requires a resolution > 192x192."

    # Extract features from the input image
    feats = get_patches_test_features(inputImgData, patch_size)
    sample_mu = np.mean(feats, axis=0)
    sample_cov = np.cov(feats.T)

    # Calculate the NIQE score
    X = sample_mu - pop_mu
    covmat = (pop_cov + sample_cov) / 2.0
    pinvmat = scipy.linalg.pinv(covmat)
    niqe_score = np.sqrt(np.dot(np.dot(X, pinvmat), X))

    return niqe_score



In [4]:
params_file_path='/kaggle/working/niqe/data/niqe_image_params.mat'

In [5]:
def evaluate_all_images(root_directory):
    # Kiểm tra thư mục gốc tồn tại
    if not os.path.exists(root_directory):
        print(f"Directory {root_directory} does not exist.")
        return

    # Lấy danh sách tất cả các thư mục con (train, test, val)
    subfolders = ['train/images', 'test/images', 'val/images']
    total_niqe = 0
    total_brisque = 0
    num_images = 0

    # Duyệt qua từng thư mục con
    for subfolder in subfolders:
        folder_path = os.path.join(root_directory, subfolder)

        if not os.path.exists(folder_path):
            print(f"Subfolder {subfolder} does not exist in {root_directory}.")
            continue

        # Lấy danh sách ảnh trong thư mục
        image_files = [f for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        # Đánh giá từng ảnh
        for image_file in image_files:
            image_path = os.path.join(folder_path, image_file)

            try:
                # Đọc ảnh
                img = io.imread(image_path)

                # Chuyển đổi hình ảnh sang grayscale (luminance channel)
                ycbcr_img = rgb2gray(img)  # rgb2gray returns a single-channel grayscale image

                # Tính NIQE
                niqe_score = niqe(ycbcr_img, params_file_path)  # NIQE expects a 2D array, not 4D
                total_niqe += niqe_score
                print(f"NIQE for {image_file}: {niqe_score:.2f}")

                # Tính BRISQUE
                img_brisque = img_as_ubyte(img)  # BRISQUE expects an 8-bit image (values from 0 to 255)
                brisque_score = BRISQUE().score(img_brisque)
                total_brisque += brisque_score
                print(f"BRISQUE for {image_file}: {brisque_score:.2f}")

                num_images += 1

            except Exception as e:
                print(f"Error processing {image_file}: {e}")

    # Tính điểm trung bình cho toàn bộ ảnh
    overall_niqe = total_niqe / num_images if num_images > 0 else None
    overall_brisque = total_brisque / num_images if num_images > 0 else None

    print("\nOverall Scores:")
    if overall_niqe is not None:
        print(f"Overall NIQE: {overall_niqe:.2f}")
    if overall_brisque is not None:
        print(f"Overall BRISQUE: {overall_brisque:.2f}")

    return overall_niqe, overall_brisque


# Define root directories for datasets
root_directories = [
    "/kaggle/input/cs406-data/Enhanced_Dataset/Enhanced_Dataset",
    "/kaggle/input/cs406-data/Enhanced_Dataset_And_Denoise/Enhanced_Dataset_And_Denoise",
    "/kaggle/input/cs406-data/enhance_dce/enhance_dce",
    "/kaggle/input/cs406-data/enhance_dce_denoise"
]

# Chạy đánh giá cho từng tập dữ liệu
for root_directory in root_directories:
    print(f"\nEvaluating dataset in: {root_directory}")
    overall_niqe, overall_brisque = evaluate_all_images(root_directory)



Evaluating dataset in: /kaggle/input/cs406-data/Enhanced_Dataset/Enhanced_Dataset
NIQE for 2015_05822.jpg: 20.59
BRISQUE for 2015_05822.jpg: 52.55
NIQE for 2015_01231.jpg: 19.99
BRISQUE for 2015_01231.jpg: 52.10
NIQE for 2015_02746.jpg: 23.07
BRISQUE for 2015_02746.jpg: 79.41
NIQE for 2015_00176.jpg: 21.87
BRISQUE for 2015_00176.jpg: 60.86
NIQE for 2015_02189.jpg: 20.88
BRISQUE for 2015_02189.jpg: 52.39
NIQE for 2015_00724.jpg: 20.69
BRISQUE for 2015_00724.jpg: 41.34
NIQE for 2015_01077.jpg: 20.80
BRISQUE for 2015_01077.jpg: 63.56
NIQE for 2015_00496.jpg: 21.34
BRISQUE for 2015_00496.jpg: 69.12
NIQE for 2015_00564.jpg: 19.20
BRISQUE for 2015_00564.jpg: 54.71
NIQE for 2015_00792.jpg: 18.17
BRISQUE for 2015_00792.jpg: 61.23
NIQE for 2015_01291.jpg: 21.31
BRISQUE for 2015_01291.jpg: 54.11
NIQE for 2015_00549.jpg: 20.28
BRISQUE for 2015_00549.jpg: 53.41
NIQE for 2015_00525.jpg: 21.35
BRISQUE for 2015_00525.jpg: 76.64
NIQE for 2015_02102.jpg: 19.66
BRISQUE for 2015_02102.jpg: 57.42
NIQE fo